In [20]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import glob
from concurrent.futures import ThreadPoolExecutor
import pickle

In [11]:
Path = "./한국근현대잡지자료_20240327/"

In [27]:
# 권호 정보 및 텍스트 데이터 추출을 위한 함수 정의
def extract_data_from_file(file_path):
    level3통합정보 = []
    권호정보 = []
    잡지정보 = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            전체데이터 = BeautifulSoup(f, features="xml")

            # level3 데이터 추출
            level3 = 전체데이터.find_all("level3")
            for loop in level3:
                level3id = loop.get("id")
                mainTitle = loop.select_one("front > biblioData > title > mainTitle").get_text()
                subjectClass = loop.select_one("front > biblioData > subjectClass")
                subjectClass = subjectClass.get_text() if subjectClass is not None else ""
                pTitle = loop.select_one("text > content > paragraph > pTitle")
                pTitle = pTitle.get_text() if pTitle is not None else ""
                paragraph = loop.find_all("paragraph")
                for paras in paragraph:
                    para = paras.get_text()
                    종합 = [level3id, mainTitle, subjectClass, pTitle, para]
                    level3통합정보.append(종합)

            # level2 데이터 추출
            level2 = 전체데이터.find_all("level2")
            for loop in level2:
                level2id = loop.get("id")
                mainTitle = loop.select_one("front > biblioData > title > mainTitle").get_text()
                publisher = loop.select_one("publication > publisher").get_text()
                dateIssued = loop.select_one("publication > dateIssued").get("date")
                종합 = [level2id, mainTitle, publisher, dateIssued]
                권호정보.append(종합)

            # level1 데이터 추출
            level1 = 전체데이터.find_all("level1")
            for loop in level1:
                level1id = loop.get("id")
                mainTitle = loop.select_one("front > biblioData > title > mainTitle").get_text()
                publisher = loop.select_one("publication > publisher").get_text()
                dateIssued = loop.select_one("publication > dateIssued").get("date")
                종합 = [level1id, mainTitle, publisher, dateIssued]
                잡지정보.append(종합)
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
    return level3통합정보, 권호정보, 잡지정보

In [28]:
# 멀티스레딩을 사용하여 데이터 추출 속도 최적화
level3통합정보 = []
권호정보 = []
잡지정보 = []

with ThreadPoolExecutor() as executor:
    file_paths = [os.path.join(root, file) for root, _, files in os.walk(Path) for file in files if '.xml' in file]
    results = executor.map(extract_data_from_file, file_paths)
    for level3, level2, level1 in results:
        level3통합정보.extend(level3)
        권호정보.extend(level2)
        잡지정보.extend(level1)

In [39]:
# df텍스트데이터 생성
columns = ["level3id", "mainTitle", "subjectClass", "pTitle", "paragraph"]
df텍스트데이터 = pd.DataFrame(level3통합정보, columns=columns)

# level2id 추출하여 df텍스트데이터에 추가
df텍스트데이터['level2id'] = df텍스트데이터['level3id'].str.extract(r'(.{11})')

# df권호정보데이터 생성
columns = ["level2id", "mainTitle_권호", "publisher", "dateIssued"]
df권호정보데이터 = pd.DataFrame(권호정보, columns=columns)

# df잡지정보데이터 생성 (불필요한 컬럼 제거)
columns = ["level1id", "mainTitle_잡지"]
df잡지정보데이터 = pd.DataFrame([[level1id, mainTitle] for level1id, mainTitle, _, _ in 잡지정보], columns=columns)

# level1id를 미리 생성하여 df텍스트데이터에 추가
df텍스트데이터['level1id'] = df텍스트데이터['level3id'].str.extract(r'(.{6})')

In [40]:
# 병합 단계 개선: 불필요한 중간 병합을 줄이고, 한 번에 병합 수행 (메모리 사용 최적화)
merged_data = pd.merge(df텍스트데이터, df권호정보데이터, on="level2id", how="left", sort=False, suffixes=('', '_권호'))
All_level통합 = pd.merge(merged_data, df잡지정보데이터, on="level1id", how="left", sort=False, suffixes=('', '_잡지'), copy=False)

In [41]:
All_level통합 = All_level통합.replace('', pd.NA) 

In [42]:
All_level통합

,level3id,mainTitle,subjectClass,pTitle,paragraph,level2id,level1id,mainTitle_권호,publisher,dateIssued,mainTitle_잡지
0,ma_061_0010_0010,卷頭一聲,<NA>,卷頭一聲,\n卷頭一聲\n又影生\n,ma_061_0010,ma_061,공제 제1호,<NA>,1920-09-10,공제
1,ma_061_0010_0020,創刊辭,<NA>,創刊辭,\n創刊辭\n,ma_061_0010,ma_061,공제 제1호,<NA>,1920-09-10,공제
2,ma_061_0010_0030,勞働萬能論,<NA>,勞働萬能論,\n勞働萬能論\n趙誠惇\n,ma_061_0010,ma_061,공제 제1호,<NA>,1920-09-10,공제
3,ma_061_0010_0040,勞働化하라,<NA>,勞働化하라,\n勞働化하라\n南相協\n,ma_061_0010,ma_061,공제 제1호,<NA>,1920-09-10,공제
4,ma_061_0010_0050,勞働問題는 社會의 根本問題이라,<NA>,勞働問題는 社會의 根本問題이라,\n勞働問題는 社會의 根本問題이라\n金明植\n,ma_061_0010,ma_061,공제 제1호,<NA>,1920-09-10,공제
...,...,...,...,...,...,...,...,...,...,...,...
588569,ma_089_0040_0110,쏘위엣트 露西亞의 붉은 新聞,<NA>,쏘위엣트 露西亞의 붉은 新聞,\n쏘위엣트 露西亞의 붉은 新聞\n金世鎔\n,ma_089_0040,ma_089,철필 제2권 제1호,<NA>,1931-01-11,철필
588570,ma_089_0040_0120,獨逸及英國 新聞界 巡訪印象,<NA>,獨逸及英國 新聞界 巡訪印象,\n獨逸及英國 新聞界 巡訪印象\n金賢準\n,ma_089_0040,ma_089,철필 제2권 제1호,<NA>,1931-01-11,철필
588571,ma_089_0040_0130,三大新聞社 人材巡禮,<NA>,三大新聞社 人材巡禮,\n三大新聞社 人材巡禮\n覆面記者\n,ma_089_0040,ma_089,철필 제2권 제1호,<NA>,1931-01-11,철필
588572,ma_089_0040_0140,新聞語辭典,<NA>,新聞語辭典,\n新聞語辭典\n編輯部\n,ma_089_0040,ma_089,철필 제2권 제1호,<NA>,1931-01-11,철필


In [43]:
All_level통합 = All_level통합.sort_values(by=['level3id']).reset_index(drop=True)
All_level통합

,level3id,mainTitle,subjectClass,pTitle,paragraph,level2id,level1id,mainTitle_권호,publisher,dateIssued,mainTitle_잡지
0,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,\n大韓自强會月報序［三編］\n,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보
1,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,＜1-4＞,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보
2,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,\n南嵩山人曰誰道日本之少文章哉此篇文字明確正大其組織極精緻極宏博至其論國家正義處尤切時病令...,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보
3,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,\n金陵\n大垣丈夫\n,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보
4,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,蓋韓國二千萬人中豈無達者也哉余嘗結交其士君子知有老練精達之人傑見有少壯有爲之人士而今則如此何...,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보
...,...,...,...,...,...,...,...,...,...,...,...
588569,ma_095_0260_0140,버드나무 그늘의 少女,<NA>,버드나무 그늘의 少女,\n버드나무 그늘의 少女\n尹影\n,ma_095_0260,ma_095,호남평론 제3권 제8호,<NA>,1937-08-15,호남평론
588570,ma_095_0260_0150,詩: 바다가에서,<NA>,詩: 바다가에서,\n詩: 바다가에서\n一石\n,ma_095_0260,ma_095,호남평론 제3권 제8호,<NA>,1937-08-15,호남평론
588571,ma_095_0260_0160,戱曲: 水害後,<NA>,戱曲: 水害後,\n戱曲: 水害後\n朴東和\n,ma_095_0260,ma_095,호남평론 제3권 제8호,<NA>,1937-08-15,호남평론
588572,ma_095_0260_0170,詩: H孃의 半生記,<NA>,詩: H孃의 半生記,\n詩: H孃의 半生記\n溝隱\n,ma_095_0260,ma_095,호남평론 제3권 제8호,<NA>,1937-08-15,호남평론


In [47]:
All_level통합[All_level통합['level3id']=='ma_001_0010_0010']

,level3id,mainTitle,subjectClass,pTitle,paragraph,level2id,level1id,mainTitle_권호,publisher,dateIssued,mainTitle_잡지
0,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,\n大韓自强會月報序［三編］\n,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보
1,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,＜1-4＞,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보
2,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,\n南嵩山人曰誰道日本之少文章哉此篇文字明確正大其組織極精緻極宏博至其論國家正義處尤切時病令...,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보
3,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,\n金陵\n大垣丈夫\n,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보
4,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,蓋韓國二千萬人中豈無達者也哉余嘗結交其士君子知有老練精達之人傑見有少壯有爲之人士而今則如此何...,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보
5,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,日에 有客이 促膝附耳고 叩之于余 曰 自强會之名稱과 及 其 趣旨 業己公佈大世矣라....,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보
6,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,\n南嶽居士\n李沂 識,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보
7,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,\n光武 十年 七月 日 會長 尹致昊\n,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보
8,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,●韓國介在乎日淸俄三國間國弱小而民未開當初受淸國壓制國勢日萎苶日淸交戰後雖脫 淸國羈絆不堪於...,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보
9,ma_001_0010_0010,大韓自强會月報序［三編］,논설,大韓自强會月報序［三編］,●會之有報 何歟오 曰 使會員讀之也니라 其 使 會員讀之 何歟오 曰 欲其求自强之道也...,ma_001_0010,ma_001,대한자강회월보 제1호,대한자강회,1906-07-31,대한자강회월보


In [45]:
All_level통합['paragraph'][2]

'\n南嵩山人曰誰道日本之少文章哉此篇文字明確正大其組織極精緻極宏博至其論國家正義處尤切時病令人神動意踴下頂門之一針撞警世之大鐘讀者須三復玩繹焉＜4＞'

In [21]:
with open("한국근현대잡지자료.pickle", "wb") as file:
    pickle.dump(All_level통합, file)

In [23]:
All_level통합['paragraph'][4]

'\n勞働問題는 社會의 根本問題이라\n金明植\n'

In [ ]:
# 병합 후 필요 없는 컬럼 제거로 메모리 사용 줄이기
del df텍스트데이터, df권호정보데이터, df잡지정보데이터, merged_data